In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('loans_lenders.csv')
df.head(10)

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
5,558112,"muc888, tristan7990, shivaun4955, sam44598568,..."
6,563395,"muc888, john38425073, trolltech4460, marianne8..."
7,575414,"muc888, dougal1825, dougal1825, jensdamsgaardv..."
8,578029,"muc888, rebecca3038, paul1853, paul1853, paul1..."
9,551251,"klaus5005, john70242429, john70242429, terry93..."


## 1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [3]:
# Tramite la funzione lambda ogni row viene convertita in una lista pulita.

df.lenders = df.lenders.apply(lambda x:x.replace(',','').split())
print('shape:', df.shape)
df.head(10)

shape: (1387432, 2)


,loan_id,lenders
0,483693,"[muc888, sam4326, camaran3922, lachheb1865, re..."
1,483738,"[muc888, nora3555, williammanashi, barbara5610..."
2,485000,"[muc888, terrystl, richardandsusan8352, sherri..."
3,486087,"[muc888, james5068, rudi5955, daniel9859, don9..."
4,534428,"[muc888, niki3008, teresa9174, mike4896, david..."
5,558112,"[muc888, tristan7990, shivaun4955, sam44598568..."
6,563395,"[muc888, john38425073, trolltech4460, marianne..."
7,575414,"[muc888, dougal1825, dougal1825, jensdamsgaard..."
8,578029,"[muc888, rebecca3038, paul1853, paul1853, paul..."
9,551251,"[klaus5005, john70242429, john70242429, terry9..."


In [4]:
# Explode ci permette di espandere ogni elemento in ogni lista mantenendo come indice 'loan_id' 
# per ogni membro della lista.

df = df.explode('lenders').reset_index(drop=True)
df.head(10)

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
5,483693,karlheinz4543
6,483693,jerrydb
7,483693,paula8951
8,483693,gmct
9,483693,amra9383


## 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [5]:
df1 = pd.read_csv('loans.csv')
print('Shape:', df1.shape)
print('')
print('Columns:', df1.columns)

Shape: (1419607, 31)

Columns: Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model'],
      dtype='object')


In [6]:
# Sono presenti molti Na nelle variabili che utilizziremo per computare la differenza

print(df1[['disburse_time','planned_expiration_time']].isnull().sum())

disburse_time                2813
planned_expiration_time    371834
dtype: int64


In [7]:
# Notiamo due Na in comune per entrambe le variabili

df1.disburse_time[(df1.disburse_time.isnull()==True)&(df1.planned_expiration_time.isnull()==True)]

423734     NaN
1129851    NaN
Name: disburse_time, dtype: object

In [8]:
# le variabili vengono convertite

df1['planned_expiration_time'] = pd.to_datetime(df1['planned_expiration_time']).dt.date
df1['disburse_time'] = pd.to_datetime(df1['disburse_time']).dt.date

In [9]:
df1['duration'] = df1['planned_expiration_time'] - df1['disburse_time']

In [10]:
df1['duration'].head()

0   54 days
1   96 days
2   37 days
3   35 days
4   58 days
Name: duration, dtype: timedelta64[ns]

In [11]:
# Gli Na sono rimasti invariati, come previsto.

print('Total duration Na:', df1['duration'].isnull().sum())

Total duration Na: 374645


## 3.Find the lenders that have funded at least twice.

In [12]:
# Questo groupby mostra quante volte ogni lender ha contribuito ad un finanziamento.

fund = df.groupby('lenders').size()
fund.head()

lenders
000               40
00000             39
0002              70
00mike00           1
0101craign0101    71
dtype: int64

In [13]:
group = fund[fund >= 2]
group.head()

lenders
000               40
00000             39
0002              70
0101craign0101    71
0132575            4
dtype: int64

In [14]:
lenders = group.reset_index().rename(columns={0:'Count'})
lenders.head()

,lenders,Count
0,000,40
1,00000,39
2,0002,70
3,0101craign0101,71
4,0132575,4


In [15]:
# Lista dei lenders con almeno due finanziamenti.

lenders.lenders

0                    000
1                  00000
2                   0002
3         0101craign0101
4                0132575
               ...      
901806           zzinnia
901807        zzivnk7094
901808          zzmcfate
901809        zzrvmf8538
901810            zzzsai
Name: lenders, Length: 901811, dtype: object

## 4. For each country, compute how many loans have involved that country as borrowers.

In [16]:
df1.columns

Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model',
       'duration'],
      dtype='object')

In [17]:
# Prestiti per ogni stato.

deff = df1.groupby('country_name')['loan_id'].size()
deff = deff.reset_index().rename(columns={'loan_id':'NumberOfLoans'})
deff.head()

,country_name,NumberOfLoans
0,Afghanistan,2337
1,Albania,3075
2,Armenia,13952
3,Azerbaijan,10172
4,Belize,218


## 5. For each country, compute the overall amount of money borrowed.

In [18]:
# Totale finanziato per ogni stato

deff = df1.groupby('country_name')['loan_amount'].sum()
deff = deff.reset_index().rename(columns={'loan_amount':'Total_loan_amount'})
deff.head()

,country_name,Total_loan_amount
0,Afghanistan,1967950.0
1,Albania,4307350.0
2,Armenia,22950475.0
3,Azerbaijan,14784625.0
4,Belize,150175.0


## 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [19]:
# Come il punto precedente ma dividendo ogni country con il totale di 'funded_amount'

deff = df1.groupby('country_name')['loan_amount'].sum() / df1.funded_amount.sum()*100
deff = deff.reset_index().rename(columns={'loan_amount':'%_Over_Total'})
deff.head()

,country_name,%_Over_Total
0,Afghanistan,0.174126
1,Albania,0.381119
2,Armenia,2.030683
3,Azerbaijan,1.308160
4,Belize,0.013288


## 7. Like the three previous points, but split for each year (with respect to disburse time).

In [20]:
df1['disburse_time'] = pd.to_datetime(df1['disburse_time'])

In [21]:
# Viene creata una nuova variabile indentificante l'anno

df1['years']=df1['disburse_time'].dt.year

In [22]:
print('Years Na:', df1.years.isnull().sum())

Years Na: 2813


In [23]:
# Conteggio dei prestiti in base allo stato e all'anno.

df1.groupby(['country_name','years'])['loan_id'].size() 

country_name  years 
Afghanistan   2007.0     408
              2008.0     370
              2009.0     678
              2010.0     632
              2011.0     247
                        ... 
Zimbabwe      2013.0     426
              2014.0    2078
              2015.0     600
              2016.0     808
              2017.0    1079
Name: loan_id, Length: 748, dtype: int64

In [24]:
# Importo totale per anno e stato.

df1.groupby(['country_name','years'])['loan_amount'].sum()

country_name  years 
Afghanistan   2007.0     194975.0
              2008.0     365375.0
              2009.0     585125.0
              2010.0     563350.0
              2011.0     245125.0
                          ...    
Zimbabwe      2013.0     678525.0
              2014.0    1311575.0
              2015.0     723625.0
              2016.0     788600.0
              2017.0    1237600.0
Name: loan_amount, Length: 748, dtype: float64

In [25]:
# Rapporto tra amount e funded per anno e stato.

df1.groupby(['country_name','years'])['loan_amount'].sum()/df1.funded_amount.sum()*100

country_name  years 
Afghanistan   2007.0    0.017252
              2008.0    0.032329
              2009.0    0.051772
              2010.0    0.049846
              2011.0    0.021689
                          ...   
Zimbabwe      2013.0    0.060037
              2014.0    0.116050
              2015.0    0.064027
              2016.0    0.069776
              2017.0    0.109504
Name: loan_amount, Length: 748, dtype: float64

## 8. For each lender, compute the overall amount of money lent.  For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [26]:
# Estraggo solo le colonne che mi servono

dfx = df1[['loan_id','loan_amount','funded_amount']]
dfx.head()

,loan_id,loan_amount,funded_amount
0,657307,125.0,125.0
1,657259,400.0,400.0
2,658010,400.0,400.0
3,659347,625.0,625.0
4,656933,425.0,425.0


In [27]:
# Merge su loan_id in modo da avere anche il nome di ogni lender

df2 = dfx.merge(df,on='loan_id')
df2.head()

In [29]:
# numero di lenders per ogni prestito

id_group = df2.groupby('loan_id').size()
id_group

loan_id
84         3
85         2
86         3
88         3
89         4
          ..
1444051    1
1444053    1
1444058    1
1444063    1
1444065    1
Length: 1387428, dtype: int64

In [30]:
# Con il groupby faccio in modo che i loan_id della tabella siano ordinati come quelli della numerosità

funded_group = df2.groupby(['loan_id','lenders'])['funded_amount'].sum()
funded_group

loan_id  lenders   
84       brooke        500.0
         michael       500.0
         ward          500.0
85       michael       500.0
         patrick       500.0
                       ...  
1444051  lauren1424     50.0
1444053  dario6858      50.0
1444058  el5018         25.0
1444063  pgs67          50.0
1444065  el5018         25.0
Name: funded_amount, Length: 27459067, dtype: float64

In [31]:
# divisione tra somma prestata e numerosità di ciascun gruppo

df3 = funded_group/id_group
df3

loan_id  lenders   
84       brooke        166.666667
         michael       166.666667
         ward          166.666667
85       michael       250.000000
         patrick       250.000000
                          ...    
1444051  lauren1424     50.000000
1444053  dario6858      50.000000
1444058  el5018         25.000000
1444063  pgs67          50.000000
1444065  el5018         25.000000
Length: 27459067, dtype: float64

In [32]:
df3 = df3.reset_index().rename(columns={0:'amount'})
df3

,loan_id,lenders,amount
0,84,brooke,166.666667
1,84,michael,166.666667
2,84,ward,166.666667
3,85,michael,250.000000
4,85,patrick,250.000000
...,...,...,...
27459062,1444051,lauren1424,50.000000
27459063,1444053,dario6858,50.000000
27459064,1444058,el5018,25.000000
27459065,1444063,pgs67,50.000000


In [33]:
# Raggruppamento per lender e somma totale

df3 = df3.groupby('lenders')['amount'].sum().reset_index().rename(columns={'amount':'Total_lent'})
df3

,lenders,Total_lent
0,000,1703.868411
1,00000,1379.750248
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117
...,...,...
1383794,zzmcfate,63381.546705
1383795,zzpaghetti9994,51.020408
1383796,zzrvmf8538,513.213719
1383797,zzzsai,267.667370


## 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [34]:
# funded_group è lo stesso del punto precendente, si procede con un raggruppamento identico ma per la 
# variabile loan_amount

amount_group = df2.groupby(['loan_id','lenders'])['loan_amount'].sum()
funded_group = df2.groupby(['loan_id','lenders'])['funded_amount'].sum()

In [35]:
df_loa = amount_group/id_group
df_fun = funded_group/id_group

In [36]:
df_loa = df_loa.reset_index().rename(columns={0:'loan_amount_each'})
df_loa

,loan_id,lenders,loan_amount_each
0,84,brooke,166.666667
1,84,michael,166.666667
2,84,ward,166.666667
3,85,michael,250.000000
4,85,patrick,250.000000
...,...,...,...
27459062,1444051,lauren1424,225.000000
27459063,1444053,dario6858,725.000000
27459064,1444058,el5018,25.000000
27459065,1444063,pgs67,50.000000


In [37]:
df_fun = df_fun.reset_index().rename(columns={0:'funded_amount_each'})
df_fun

,loan_id,lenders,funded_amount_each
0,84,brooke,166.666667
1,84,michael,166.666667
2,84,ward,166.666667
3,85,michael,250.000000
4,85,patrick,250.000000
...,...,...,...
27459062,1444051,lauren1424,50.000000
27459063,1444053,dario6858,50.000000
27459064,1444058,el5018,25.000000
27459065,1444063,pgs67,50.000000


In [38]:
tot = df_loa.merge(df_fun,on=['loan_id','lenders'])

In [86]:
tot['difference']= tot.loan_amount_each - tot.funded_amount_each

In [87]:
tot_clean = tot[tot.difference!=0]

In [88]:
tot_clean.head()

,loan_id,lenders,loan_amount_each,funded_amount_each,difference
2560,726,acusticthoughts,50.0,48.4375,1.5625
2561,726,chris3300,50.0,48.4375,1.5625
2562,726,chris3376,50.0,48.4375,1.5625
2563,726,craig5654,50.0,48.4375,1.5625
2564,726,fpierfed,50.0,48.4375,1.5625


In [89]:
df_lenders= pd.read_csv('lenders.csv')
df_lenders.dropna(subset=['country_code',],inplace=True)
df_lenders.dropna(subset=['permanent_name'],inplace=True)
df_lenders.rename(columns={'permanent_name':'lenders'}, inplace=True)
df_lenders =  df_lenders[['lenders', 'country_code']]

In [91]:
fmerge = df.merge(df_lenders, on='lenders')
fmerge.dropna(inplace=True)
fmerge.head()

,loan_id,lenders,country_code
0,483693,muc888,US
1,483738,muc888,US
2,485000,muc888,US
3,486087,muc888,US
4,534428,muc888,US


In [92]:
mmerge = tot_clean.merge(fmerge,on=['loan_id','lenders'])

In [93]:
mmerge.head()

,loan_id,lenders,loan_amount_each,funded_amount_each,difference,country_code
0,726,acusticthoughts,50.0,48.4375,1.5625,US
1,726,chris3300,50.0,48.4375,1.5625,US
2,726,chris3376,50.0,48.4375,1.5625,US
3,726,craig5654,50.0,48.4375,1.5625,US
4,726,fpierfed,50.0,48.4375,1.5625,US


In [94]:
abibi = mmerge.groupby('country_code')['difference'].sum().reset_index()
abibi.head()

,country_code,difference
0,AD,63.583333
1,AE,48736.111556
2,AF,3819.582746
3,AL,118.452412
4,AM,1627.617004


In [156]:
df1 = pd.read_csv('C:/Users/giaco/Desktop/Dataset Esame CS/loans.csv')

In [157]:
dfb = df1[['country_code','loan_amount','funded_amount']]

In [158]:
dfb['diff'] = df1.loan_amount-df1.funded_amount

C:\Users\giaco\Anaconda4\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [159]:
dfb.groupby('country_code')['diff'].sum()

country_code
AF       3300.0
AL     314875.0
AM    2665800.0
AZ     593275.0
BA          0.0
        ...    
XK     220700.0
YE     103975.0
ZA          0.0
ZM      65725.0
ZW      33325.0
Name: diff, Length: 95, dtype: float64

# 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [101]:
df6 = pd.read_csv('C:/Users/giaco/Desktop/Dataset Esame CS/country_stats.csv')
df6.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh


In [102]:
abibi.country_code.nunique()

191

In [103]:
df6.country_code.nunique()

173

In [104]:
df7 = df6[['country_code','population']].merge(abibi,on='country_code')
df7.head(10)

,country_code,population,difference
0,IN,1339180127,1.874844e+04
1,NG,190886311,7.002354e+02
2,MX,129163276,4.522973e+04
3,PK,197015955,1.081646e+03
4,BD,164669751,3.918846e+02
5,CD,81339988,2.219515e+01
6,US,324459463,2.165696e+07
7,CN,1409517397,1.759343e+04
8,ET,104957438,6.706940e+03
9,ID,263991379,9.024395e+03


In [105]:
df7.country_code.nunique()

151

In [108]:
df7['ratio']= df7.difference/df7.population
df7.head(20)

,country_code,population,difference,ratio
0,IN,1339180127,1.874844e+04,1.399994e-05
1,NG,190886311,7.002354e+02,3.668338e-06
2,MX,129163276,4.522973e+04,3.501748e-04
3,PK,197015955,1.081646e+03,5.490142e-06
4,BD,164669751,3.918846e+02,2.379821e-06
5,CD,81339988,2.219515e+01,2.728689e-07
6,US,324459463,2.165696e+07,6.674781e-02
7,CN,1409517397,1.759343e+04,1.248188e-05
8,ET,104957438,6.706940e+03,6.390152e-05
9,ID,263991379,9.024395e+03,3.418443e-05


In [109]:
df7[df7.ratio == df7.ratio.max()]

,country_code,population,difference,ratio
141,NO,5305383,932265.20019,0.175721


## 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [110]:
df6[['country_code','population','population_below_poverty_line']].head()

,country_code,population,population_below_poverty_line
0,IN,1339180127,21.9
1,NG,190886311,70.0
2,MX,129163276,46.2
3,PK,197015955,29.5
4,BD,164669751,31.5


In [111]:
df6['population_below_poverty_line'] = df6.population_below_poverty_line/100
df6['population_not_below_poverty_line'] = 1 - df6.population_below_poverty_line

In [114]:
df8 = df6[['country_code','population','population_not_below_poverty_line']].merge(abibi[['country_code','difference']],on='country_code')

In [115]:
df8.head()

,country_code,population,population_not_below_poverty_line,difference
0,IN,1339180127,0.781,18748.435665
1,NG,190886311,0.300,700.235426
2,MX,129163276,0.538,45229.725459
3,PK,197015955,0.705,1081.645655
4,BD,164669751,0.685,391.884560


In [116]:
df8['ratio'] = df8.difference/(df8.population*df8.population_not_below_poverty_line)

In [119]:
df8[df8.ratio == df8.ratio.max()]

,country_code,population,population_not_below_poverty_line,difference,ratio
6,US,324459463,0.849,2.165696e+07,0.078619


## 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [131]:
df1 = pd.read_csv('C:/Users/giaco/Desktop/Dataset Esame CS/loans.csv',nrows=500000)

In [132]:
# Vengono rimossi i NA per entrambe le variabili

df1.dropna(subset=['planned_expiration_time'],inplace=True)
df1.dropna(subset=['disburse_time'],inplace=True)

In [133]:
df1['planned_expiration_time'] = pd.to_datetime(df1['planned_expiration_time'])
df1['disburse_time'] = pd.to_datetime(df1['disburse_time'])

In [134]:
df1['year_start']= df1['disburse_time'].dt.year
df1['year_end']=df1['planned_expiration_time'].dt.year
df1['days_first_year']= 365 - df1['disburse_time'].dt.dayofyear
df1['days_last_year']=df1['planned_expiration_time'].dt.dayofyear
df1['difference_years']=df1['year_end']-df1['year_start']
df1['2011']=0 
df1['2012']=0
df1['2013']=0
df1['2014']=0
df1['2015']=0
df1['2016']=0
df1['2017']=0
df1['2018']=0

In [136]:
atribute = ['loan_amount', '2011', '2012', '2013', '2014','2015', '2016', 
            '2017', '2018', 'difference_years', 'days_first_year', 'days_last_year', 'year_start', 'year_end']
df1 = df1[atribute][(df1.difference_years != -1)]

In [137]:
df1.head()

,loan_amount,2011,2012,2013,2014,2015,2016,2017,2018,difference_years,days_first_year,days_last_year,year_start,year_end
0,125.0,0,0,0,0,0,0,0,0,1,9,45,2013,2014
1,400.0,0,0,0,0,0,0,0,0,1,11,85,2013,2014
2,400.0,0,0,0,0,0,0,0,0,0,356,46,2014,2014
3,625.0,0,0,0,0,0,0,0,0,0,348,52,2014,2014
4,425.0,0,0,0,0,0,0,0,0,1,14,44,2013,2014


In [140]:
def loan(x):
    if x['difference_years'] == 0:
        x[str(int(x['year_start']))]= x['loan_amount']
    elif x['difference_years']== 1:
        x[str(int(x['year_start']))]= (x['days_first_year']*x['loan_amount'])/((x['days_first_year'])+ x['days_last_year']+ ((x['difference_years']-1)*365))
        x[str(int(x['year_end']))]= x['loan_amount'] - x[str(int(x['year_start']))]
    elif x['difference_years'] > 1:
        x[str(int(x['year_start']))]= x['days_first_year']*x['loan_amount'] / (x['days_first_year'] + x['days_last_year'] + ((x['difference_years']-1)*365))
        x[str(int(x['year_end']))]=(x['days_last_year']*x['loan_amount'])/((x['days_first_year'])+ x['days_last_year']+ ((x['difference_years']-1)*365))
        for y in range(1,int(x['difference_years'])):
            x[str(int(x['year_start']+y))] = (365*x['loan_amount'])/((x['days_first_year'])+ x['days_last_year']+ ((x['difference_years']-1)*365))
    return x

In [141]:
df1 = df1.apply(loan,axis=1)

In [142]:
df1.head()

,loan_amount,2011,2012,2013,2014,2015,2016,2017,2018,difference_years,days_first_year,days_last_year,year_start,year_end
0,125.0,0.0,0.0,20.833333,104.166667,0.0,0.0,0.0,0.0,1.0,9.0,45.0,2013.0,2014.0
1,400.0,0.0,0.0,45.833333,354.166667,0.0,0.0,0.0,0.0,1.0,11.0,85.0,2013.0,2014.0
2,400.0,0.0,0.0,0.000000,400.000000,0.0,0.0,0.0,0.0,0.0,356.0,46.0,2014.0,2014.0
3,625.0,0.0,0.0,0.000000,625.000000,0.0,0.0,0.0,0.0,0.0,348.0,52.0,2014.0,2014.0
4,425.0,0.0,0.0,102.586207,322.413793,0.0,0.0,0.0,0.0,1.0,14.0,44.0,2013.0,2014.0


In [143]:
just_years = '2011 2012 2013 2014 2015 2016 2017 2018'.split()

In [144]:
df1[just_years].apply(lambda x: int(x.sum())).reset_index().rename(columns={'index':'Year',0:'Total'})

,Year,Total
0,2011,220841
1,2012,40905081
2,2013,45871790
3,2014,55407433
4,2015,54853023
5,2016,53768993
6,2017,61009747
7,2018,2081788
